In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/titanicdata.csv


# Feature Construction

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df= pd.read_csv('/kaggle/input/titanic/titanicdata.csv')[['Age', 'Pclass', 'SibSp', 'Parch', 'Survived']]

In [4]:
df.dropna(inplace= True)

In [5]:
df.head()

,Age,Pclass,SibSp,Parch,Survived
0,22.0,3,1,0,0
1,38.0,1,1,0,1
2,26.0,3,0,0,1
3,35.0,1,1,0,1
4,35.0,3,0,0,0


In [6]:
df.tail()

,Age,Pclass,SibSp,Parch,Survived
885,39.0,3,0,5,0
886,27.0,2,0,0,0
887,19.0,1,0,0,1
889,26.0,1,0,0,1
890,32.0,3,0,0,0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 714 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Age       714 non-null    float64
 1   Pclass    714 non-null    int64  
 2   SibSp     714 non-null    int64  
 3   Parch     714 non-null    int64  
 4   Survived  714 non-null    int64  
dtypes: float64(1), int64(4)
memory usage: 33.5 KB


In [8]:
df['Age'].value_counts()

Age
24.00    30
22.00    27
18.00    26
28.00    25
30.00    25
         ..
24.50     1
0.67      1
0.42      1
34.50     1
74.00     1
Name: count, Length: 88, dtype: int64

In [9]:
df.describe()

,Age,Pclass,SibSp,Parch,Survived
count,714.000000,714.000000,714.000000,714.000000,714.000000
mean,29.699118,2.236695,0.512605,0.431373,0.406162
std,14.526497,0.838250,0.929783,0.853289,0.491460
min,0.420000,1.000000,0.000000,0.000000,0.000000
25%,20.125000,1.000000,0.000000,0.000000,0.000000
50%,28.000000,2.000000,0.000000,0.000000,0.000000
75%,38.000000,3.000000,1.000000,1.000000,1.000000
max,80.000000,3.000000,5.000000,6.000000,1.000000


In [10]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LogisticRegression


In [11]:
X = df.iloc[:,0:4]
y = df.iloc[:,-1]

In [12]:
X.head()

,Age,Pclass,SibSp,Parch
0,22.0,3,1,0
1,38.0,1,1,0
2,26.0,3,0,0
3,35.0,1,1,0
4,35.0,3,0,0


In [13]:
y

0      0
1      1
2      1
3      1
4      0
      ..
885    0
886    0
887    1
889    1
890    0
Name: Survived, Length: 714, dtype: int64

In [14]:
np.mean(cross_val_score(LogisticRegression(), X,y, cv=20, scoring= 'accuracy'))

np.float64(0.6933333333333332)

# Applying Feature Construction

In [15]:
X['Family_size']= X['SibSp'] + X['Parch'] + 1

In [16]:
X.head()

,Age,Pclass,SibSp,Parch,Family_size
0,22.0,3,1,0,2
1,38.0,1,1,0,2
2,26.0,3,0,0,1
3,35.0,1,1,0,2
4,35.0,3,0,0,1


In [17]:
def new_fun(num):
    if num == 1:
        #alone
        return 0
    elif num > 1 and 4 < num:
        #small family
        return 1
    else:
        #large family
        return 2

In [18]:
new_fun(4)

2

In [19]:
X['Family_type'] = X['Family_size'].apply(new_fun)

In [20]:
X.head()

,Age,Pclass,SibSp,Parch,Family_size,Family_type
0,22.0,3,1,0,2,2
1,38.0,1,1,0,2,2
2,26.0,3,0,0,1,0
3,35.0,1,1,0,2,2
4,35.0,3,0,0,1,0


In [21]:
X.drop(columns= ['SibSp', 'Parch', 'Family_size'], inplace= True)

In [22]:
X.head()

,Age,Pclass,Family_type
0,22.0,3,2
1,38.0,1,2
2,26.0,3,0
3,35.0,1,2
4,35.0,3,0


In [23]:
np.mean(cross_val_score(LogisticRegression(),X,y,scoring= 'accuracy', cv= 20))

np.float64(0.7312698412698414)